In [26]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import sys, random
import pandas as pd
from konlpy.tag import Twitter
import re

In [27]:
fp = codecs.open('투캅스(샘플).txt', 'r', 'utf-8')
lines = fp.readlines()
fp.close()


In [28]:
# with open('투캅스(샘플).txt', 'rb') as fp2:  # codecs를 안쓰는 법???
#     lines2 = fp2.read()
# lines2 = lines2.decode('utf-8')
# # print(lines)
# lines2 = ''.join(lines2).split('\n')
# print(lines2)

In [29]:
results = []

for line in lines:
    cleantext = BeautifulSoup(line, 'lxml').text # 전에는 lxml이 아닌 html.parser 사용
    # lxml을 사용하면 html.parser보다 빠르지만 따로 설치가 필요, parser종류 중 하나
    # lxml은 c언어 라이브러리? 더 유연함.
    cleantext = re.sub('[\r\n!@#$]', '', cleantext)
    results.append(cleantext)

all_text = ' '.join(results)
text = all_text
print('훈련 데이터', len(text))

훈련 데이터 67523


In [30]:
text_split = text.split(' ')
chars = sorted(list(set(text_split))) # 단어 별 중복 제거하고 글자 순으로 나열
print('단어의 종류', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 5
step = 1
sentences = []
next_chars = []

for i in range(0, len(text_split) - maxlen, step): # 전의 질병패턴같이 중복해서 학습
    sentences.append(' '.join(text_split[i: i + maxlen]))
    next_chars.append(text_split[i + maxlen])
    
# sentences == 5단어로 이뤄진 문장들의 list
print('학습할 문장의 수:', len(sentences))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence.split(' ')):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print(X.shape, y.shape)

단어의 종류 7696
학습할 문장의 수: 15229
(15229, 5, 7696) (15229, 7696)


In [56]:
print('lstm 모델을 만들어 본다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) # 128 메모리 셀을 가진 LSTM 레이어 생성
model.add(Dense(len(chars)))  # 완전 연결 레이어(Dense) 레이어 == 전결합층이라 불림
# dense : 입력과 출력을 모두 연결해줌.
model.add(Activation('softmax'))
# softmax를 사용하면 0에서 1사이이며, 모든 확률의 합은 1이다.
# 출력층에서 확률표현을 위해 사용됨.
optimizer = RMSprop(lr=0.015)
# RNN의 옵티마이저로 많이 사용됨.
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
# crossentropy == log씌운 미분식. loss function.

def sample(preds, temp=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
#     print(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

lstm 모델을 만들어 본다...


In [57]:
generated_all = []
for iteration in range(1, 8):
    print()
    print('반복번호 = ', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    for diversity in [1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6]:
        print()
        print('추출옵션 : ', diversity)
        generated =''
        sentence ='범인을 빨리 잡아야지.'
        generated += sentence
        print('질문시작패턴 = "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(1):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence.split(' ')):
                x[0, t, char_indices[char]] = 1
            
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            
            generated += " " + next_char
            sentence = ' '.join(sentence.split(' ')[1:]) + ' ' + next_char
            sys.stdout.write(' ' + next_char)
            sys.stdout.flush()
            
            if next_char in ['.']:
                generated_all.append(generated)
                break
        print()
            


반복번호 =  1
Epoch 1/1
15229/15229 [==============================] - 25s 2ms/step - loss: 8.6716

추출옵션 :  1.4
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지.[6.03267614e-04 1.08823278e-04 1.03647094e-04 1.03203156e-04
 1.07926167e-04 1.05922299e-04 1.03616175e-04 1.17534678e-04
 1.18112739e-04 1.14403973e-04 1.10136400e-04 8.58910783e-05
 1.14610320e-04 1.18647586e-04 1.08863926e-04 8.11474252e-05
 1.16966984e-04 9.71793273e-05 1.01494610e-04 1.09117022e-04
 1.01839779e-04 9.18447632e-05 8.75848937e-05 8.17079852e-05
 1.16959451e-04 1.01199170e-04 1.08857566e-04 1.13007383e-04
 1.10436811e-04 1.12501352e-04 1.04278574e-04 8.38533031e-05
 1.64252721e-04 1.07341610e-04 2.37816848e-04 1.06456261e-04
 1.04032657e-04 7.97094788e-05 1.10986524e-04 7.45600976e-05
 1.12992584e-04 7.21784665e-05 8.64688904e-05 1.23030042e-04
 1.06140916e-04 7.58598672e-05 1.07642982e-04 9.97744503e-05
 1.03570643e-04 8.95093477e-05 1.12365063e-04 1.10898344e-04
 1.14885818e-04 1.13758982e-04 1.09396598e-04 1.08232644e-04
 

 최형사를

추출옵션 :  1.6
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지.[5.03579246e-04 1.12526068e-04 1.07828621e-04 1.07424395e-04
 1.11713967e-04 1.09896924e-04 1.07800475e-04 1.20369604e-04
 1.20887448e-04 1.17559446e-04 1.13713252e-04 9.14799907e-05
 1.17744960e-04 1.21366298e-04 1.12562844e-04 8.70436241e-05
 1.19860737e-04 1.01917489e-04 1.05866650e-04 1.12791794e-04
 1.06181616e-04 9.70050095e-05 9.30565870e-05 8.75695267e-05
 1.19853983e-04 1.05596955e-04 1.12557090e-04 1.16302762e-04
 1.13984602e-04 1.15846945e-04 1.08403240e-04 8.95780732e-05
 1.61322545e-04 1.11184348e-04 2.23014887e-04 1.10381520e-04
 1.08179518e-04 8.56924948e-05 1.14480899e-04 8.08285379e-05
 1.16289436e-04 7.85648450e-05 9.20182488e-05 1.25279884e-04
 1.10095366e-04 8.20601170e-05 1.11457441e-04 1.04295000e-04
 1.07759024e-04 9.48432454e-05 1.15724137e-04 1.14401308e-04
 1.17992576e-04 1.16979308e-04 1.13044622e-04 1.11991497e-04
 1.08700459e-04 1.06143238e-04 1.10595737e-04 1.09947668e-04
 1.29689373e-04 1.09798649e-04

 아까부터

추출옵션 :  1.8
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지.[4.36366071e-04 1.15172564e-04 1.10888784e-04 1.10519198e-04
 1.14433422e-04 1.12777452e-04 1.10863055e-04 1.22281626e-04
 1.22749132e-04 1.19740709e-04 1.16252028e-04 9.58106745e-05
 1.19908655e-04 1.23181236e-04 1.15206022e-04 9.16692218e-05
 1.21822007e-04 1.05468527e-04 1.09093491e-04 1.15414289e-04
 1.09381946e-04 1.00937412e-04 9.72770405e-05 9.21613678e-05
 1.21815904e-04 1.08846420e-04 1.15200787e-04 1.18602252e-04
 1.16498580e-04 1.18188981e-04 1.11413897e-04 9.40379870e-05
 1.58638474e-04 1.13951063e-04 2.11553775e-04 1.13219386e-04
 1.11209487e-04 9.04032982e-05 1.16949354e-04 8.58274079e-05
 1.18590172e-04 8.36874336e-05 9.63116127e-05 1.26705750e-04
 1.12958450e-04 8.69888708e-05 1.14199819e-04 1.07652693e-04
 1.10825163e-04 9.89354509e-05 1.18077604e-04 1.16877078e-04
 1.20132777e-04 1.19215319e-04 1.15644221e-04 1.14686086e-04
 1.11685388e-04 1.09346804e-04 1.13414675e-04 1.12823738e-04
 1.30662254e-04 1.12687802e-04

 과장:(괜히

추출옵션 :  2.0
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지.[3.88495250e-04 1.17147755e-04 1.13218839e-04 1.12879165e-04
 1.16470901e-04 1.14952887e-04 1.13195195e-04 1.23635993e-04
 1.24061327e-04 1.21321419e-04 1.18135471e-04 9.92641307e-05
 1.21474554e-04 1.24454309e-04 1.17178382e-04 9.53939926e-05
 1.23217675e-04 1.08225706e-04 1.11567780e-04 1.17369015e-04
 1.11833243e-04 1.04031961e-04 1.00630389e-04 9.58547979e-05
 1.23212120e-04 1.11340347e-04 1.17173591e-04 1.20282785e-04
 1.18360940e-04 1.19905505e-04 1.13701257e-04 9.76096661e-05
 1.56274280e-04 1.16028954e-04 2.02487649e-04 1.15358221e-04
 1.13513493e-04 9.42075443e-05 1.18773042e-04 8.99048731e-05
 1.20271760e-04 8.78848590e-05 9.97311037e-05 1.27654617e-04
 1.15118915e-04 9.09991131e-05 1.16256892e-04 1.10240772e-04
 1.13160374e-04 1.02173102e-04 1.19803806e-04 1.18706977e-04
 1.21678879e-04 1.20842219e-04 1.17579437e-04 1.16702321e-04
 1.13950585e-04 1.11800905e-04 1.15537287e-04 1.14995348e-04
 1.31236599e-04 1.14870643e-

 다짜고짜

추출옵션 :  2.2
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지.[3.52915963e-04 1.18672895e-04 1.15049068e-04 1.14735240e-04
 1.18049398e-04 1.16649851e-04 1.15027227e-04 1.24633336e-04
 1.25023062e-04 1.22510386e-04 1.19582161e-04 1.02082209e-04
 1.22650957e-04 1.25383036e-04 1.18701100e-04 9.84575128e-05
 1.24249920e-04 1.10427080e-04 1.13522826e-04 1.18876641e-04
 1.13768358e-04 1.06530086e-04 1.03358728e-04 9.88897852e-05
 1.24244828e-04 1.13312426e-04 1.18696688e-04 1.21556549e-04
 1.19789625e-04 1.21209885e-04 1.15494633e-04 1.00534271e-04
 1.54215306e-04 1.17642113e-04 1.95168729e-04 1.17023716e-04
 1.15321234e-04 9.73436531e-05 1.20168725e-04 9.32933883e-05
 1.21546419e-04 9.13858366e-05 1.02518688e-04 1.28310723e-04
 1.16803002e-04 9.43250760e-05 1.17852191e-04 1.12294649e-04
 1.14995058e-04 1.04798221e-04 1.21116422e-04 1.20107959e-04
 1.22838491e-04 1.22070402e-04 1.19070376e-04 1.18262612e-04
 1.15724847e-04 1.13738451e-04 1.17188841e-04 1.16689020e-04
 1.31579674e-04 1.16573977e-04

...

 같은데요?

추출옵션 :  2.2
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지.[2.70678861e-04 1.04602392e-04 1.03165451e-04 9.43186440e-05
 9.69491640e-05 1.19908647e-04 1.00181766e-04 1.23850439e-04
 9.89040036e-05 1.03964408e-04 1.23547285e-04 1.22861693e-04
 1.09333542e-04 1.09193294e-04 1.27212728e-04 1.25518047e-04
 1.01690116e-04 1.14188957e-04 9.23100014e-05 1.22981412e-04
 1.08336801e-04 1.56510496e-04 1.24930554e-04 1.06176951e-04
 1.03033788e-04 1.02093833e-04 1.18889448e-04 9.50330601e-05
 1.11154547e-04 1.10710572e-04 9.42720470e-05 1.10310988e-04
 1.68220459e-04 1.15236046e-04 2.27018566e-04 1.26229173e-04
 1.11690906e-04 1.03837032e-04 1.13467610e-04 1.02919781e-04
 1.23466548e-04 1.15125957e-04 1.00299893e-04 1.05525975e-04
 1.08450306e-04 1.14396844e-04 1.18499934e-04 1.43100049e-04
 1.16138670e-04 1.11935889e-04 1.11067247e-04 9.07133383e-05
 1.03085389e-04 1.08820965e-04 1.11919533e-04 1.21198616e-04
 9.94045133e-05 9.06801936e-05 1.07485259e-04 1.22423995e-04
 1.13566249e-04 9.70492989e-0

 양복들:(중국말로)예.

추출옵션 :  2.4
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지.[2.55469533e-04 1.06865073e-04 1.05518609e-04 9.71934628e-05
 9.96754104e-05 1.21116208e-04 1.02717773e-04 1.24760965e-04
 1.01516200e-04 1.06267452e-04 1.24481001e-04 1.23847646e-04
 1.11287555e-04 1.11156690e-04 1.27862244e-04 1.26299983e-04
 1.04134543e-04 1.15809646e-04 9.52943892e-05 1.23958265e-04
 1.10357193e-04 1.54615898e-04 1.25757987e-04 1.08338720e-04
 1.05395159e-04 1.04513450e-04 1.20172198e-04 9.78680926e-05
 1.12985471e-04 1.12571721e-04 9.71494461e-05 1.12199222e-04
 1.65187894e-04 1.16782729e-04 2.17426378e-04 1.26955756e-04
 1.13485132e-04 1.06148097e-04 1.15138849e-04 1.05288252e-04
 1.24406432e-04 1.16680456e-04 1.02828792e-04 1.07729686e-04
 1.10463174e-04 1.16002899e-04 1.19811242e-04 1.42427036e-04
 1.17620967e-04 1.13713286e-04 1.12904125e-04 9.37823670e-05
 1.05443544e-04 1.10809202e-04 1.13698055e-04 1.22310054e-04
 1.01987019e-04 9.37509560e-05 1.09561794e-04 1.23443143e-04
 1.15230596e-04 9.9769

 뛰어내린다.

추출옵션 :  2.6
질문시작패턴 = "범인을 빨리 잡아야지."
범인을 빨리 잡아야지.[2.43138096e-04 1.08759029e-04 1.07493497e-04 9.96404604e-05
 1.01986883e-04 1.22081474e-04 1.04857006e-04 1.25468795e-04
 1.03724251e-04 1.08197481e-04 1.25208879e-04 1.24620709e-04
 1.12907154e-04 1.12784592e-04 1.28345028e-04 1.26896813e-04
 1.06191326e-04 1.17135617e-04 9.78419750e-05 1.24723453e-04
 1.12035579e-04 1.52948003e-04 1.26394062e-04 1.10142696e-04
 1.07377405e-04 1.06547945e-04 1.21202871e-04 1.00278704e-04
 1.14496342e-04 1.14109258e-04 9.95988059e-05 1.13760672e-04
 1.62576711e-04 1.18043839e-04 2.09513885e-04 1.27504880e-04
 1.14963656e-04 1.08085302e-04 1.16509190e-04 1.07276862e-04
 1.25139642e-04 1.17948410e-04 1.04961615e-04 1.09571027e-04
 1.12134892e-04 1.17316034e-04 1.20866785e-04 1.41783374e-04
 1.18825737e-04 1.15176987e-04 1.14420248e-04 9.64080699e-05
 1.07422906e-04 1.12459097e-04 1.15162747e-04 1.23191849e-04
 1.04168227e-04 9.63782629e-05 1.11289991e-04 1.24244944e-04
 1.16594885e-04 1.02076008e-

 차고는

반복번호 =  3
Epoch 1/1
12416/15229 [=======================>......] - ETA: 4s - loss: 6.9559

KeyboardInterrupt: 

In [55]:
np.set_printoptions(threshold=1000000)
zz = np.zeros((1, maxlen, len(chars)))
print(zz.shape)
zz[0,0,0] = 1
zz[0,1,41] = 1
zz[0,2,23] = 1
alpha = model.predict(zz)[0]
# def sample(preds, temp=1.0):
#     preds = np.asarray(preds).astype('float64')
#     preds = np.log(preds) / temp
#     exp_preds = np.exp(preds)
#     preds = exp_preds / np.sum(exp_preds)
#     probas = np.random.multinomial(1, preds, 1)
#     return np.argmax(probas)
alpha2 = np.log(alpha)
alpha3 = np.exp(alpha2) / 1.4
alpha4 = alpha3 / np.sum(alpha3)
alpha5 = np.random.multinomial(1,alpha4, 1)
alpha6 = np.argmax(alpha5)
print(alpha4)

(1, 5, 7696)
[0.00012996 0.00013001 0.00012984 0.00012983 0.00012985 0.00013
 0.00012998 0.00012995 0.00013001 0.00012999 0.00012996 0.00012983
 0.00013    0.00013007 0.00012995 0.00013003 0.00013007 0.00012989
 0.00012998 0.0001299  0.00012995 0.00012989 0.0001299  0.00013001
 0.00012985 0.00012999 0.0001299  0.00012988 0.00012999 0.00013001
 0.00012991 0.00012995 0.00012977 0.00012983 0.00012989 0.00012988
 0.00012985 0.00013003 0.00012999 0.00012993 0.00012994 0.00012996
 0.00012997 0.00012997 0.00012993 0.0001299  0.00012999 0.0001299
 0.00012993 0.00013001 0.00012987 0.00012992 0.00012993 0.00012995
 0.00012996 0.0001299  0.00012994 0.00012996 0.00012994 0.00012992
 0.00013002 0.00012985 0.00012998 0.00013005 0.00012996 0.00013006
 0.00012992 0.00012992 0.00012994 0.00013003 0.00012994 0.00012993
 0.00013005 0.00012992 0.00012981 0.00013004 0.00012998 0.00012995
 0.00012992 0.0001299  0.00012998 0.00012997 0.0001299  0.00012994
 0.00013004 0.00012987 0.00012991 0.00012997 0.000129